In [1]:
# Import libraries
import pandas as pd
import numpy as np
import pandas_datareader
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import MonthEnd
import re

import warnings
warnings.filterwarnings('ignore')

## Data Cleaning
### Data Cleaning Dataset A

In [2]:
# Read files
df1 = pd.read_csv("DatasetA.csv")

In [3]:
df1.head(3)

,Primary Company,Campaign ID,Category,Incentive Text,Earliest Month,Latest Month,Observation ID
0,Ent Credit Union,20200404-0111266,Checking,$300 bonus when opening a new checking account...,2020-03,2020-03,3
1,Ent Credit Union,20200404-0111266,Checking,$200 bonus when opening a new checking account...,2020-03,2020-03,5
2,Ent Credit Union,20201215-011846,Checking,$100 when referring a friend or family member ...,2020-11,2021-02,9


In [4]:
## Extract information from "Incentive Text" Column

# Parse the bonus amount
def parse_bonus(incentive):
    match = re.search(r"\$\d+", incentive)
    if match:
        bonus = int(match.group().replace("$", ""))
    else:
        bonus = 0
    return bonus

df1["Bonus"] = df1["Incentive Text"].apply(parse_bonus)

In [5]:
# correct misspecified cells manually (since there is no "$" in the text)
df1.at[195,"Bonus"] = 4000
df1.at[3868,"Bonus"] = 10
df1.at[4006,"Bonus"] = 25
df1.at[4180,"Bonus"] = 100
df1.at[6337,"Bonus"] = 50
df1.at[9887,"Bonus"] = 10

In [6]:
# Parse the required deposit amount
def parse_deposit(incentive):
    matches = re.findall(r"\$[\d,]+", incentive)
    if len(matches)>=2:
        deposit = int(matches[1].replace("$", "").replace(",", ""))
    else:
        deposit = 0
    return deposit

df1["Deposit"] = df1["Incentive Text"].apply(parse_deposit)

In [7]:
print(df1["Bonus"].describe())

count    11196.000000
mean       164.894337
std        157.303539
min          0.000000
25%         25.000000
50%        150.000000
75%        250.000000
max       5000.000000
Name: Bonus, dtype: float64


In [8]:
df1["Bank"] = df1["Primary Company"].str.title()

df1["Earliest Quarter"] = pd.to_datetime(df1["Earliest Month"]).dt.to_period("Q")
df1["Latest Quarter"] = pd.to_datetime(df1["Latest Month"]).dt.to_period("Q")

In [9]:
df1 = df1[["Bank","Earliest Quarter","Latest Quarter","Observation ID","Bonus","Deposit"]]
df1

,Bank,Earliest Quarter,Latest Quarter,Observation ID,Bonus,Deposit
0,Ent Credit Union,2020Q1,2020Q1,3,300,100
1,Ent Credit Union,2020Q1,2020Q1,5,200,10000
2,Ent Credit Union,2020Q4,2021Q1,9,100,0
3,Ent Credit Union,2021Q4,2021Q4,21,200,0
4,Ent Credit Union,2022Q3,2022Q3,23,200,100
...,...,...,...,...,...,...
11191,Bank Of Blue Valley,2019Q3,2019Q3,2286122,350,0
11192,Minnesota Bank & Trust,2019Q3,2019Q3,2286233,350,0
11193,Minnesota Bank & Trust,2019Q3,2019Q3,2286269,350,0
11194,Addition Financial,2019Q3,2019Q3,2286284,300,500


In [10]:
# Create a variable Incentive
df1["Incentive"] = np.where(df1["Bonus"]>=25, 1, 0)
df1

,Bank,Earliest Quarter,Latest Quarter,Observation ID,Bonus,Deposit,Incentive
0,Ent Credit Union,2020Q1,2020Q1,3,300,100,1
1,Ent Credit Union,2020Q1,2020Q1,5,200,10000,1
2,Ent Credit Union,2020Q4,2021Q1,9,100,0,1
3,Ent Credit Union,2021Q4,2021Q4,21,200,0,1
4,Ent Credit Union,2022Q3,2022Q3,23,200,100,1
...,...,...,...,...,...,...,...
11191,Bank Of Blue Valley,2019Q3,2019Q3,2286122,350,0,1
11192,Minnesota Bank & Trust,2019Q3,2019Q3,2286233,350,0,1
11193,Minnesota Bank & Trust,2019Q3,2019Q3,2286269,350,0,1
11194,Addition Financial,2019Q3,2019Q3,2286284,300,500,1


In [11]:
# Transform the df1 from 'start-end' to 'all quarter' format
df11 = df1.copy()
df11['Earliest Quarter'] = df11['Earliest Quarter'].astype(str)
df11['Latest Quarter'] = df11['Latest Quarter'].astype(str)

# compute the number of quarters between start and end quarter
def quarter_difference(start_date, end_date):
    start_year, start_quarter = start_date.split("Q")
    end_year, end_quarter = end_date.split("Q")
    
    start_year, end_year = int(start_year), int(end_year)
    start_quarter, end_quarter = int(start_quarter), int(end_quarter)
    
    return (end_year-start_year)*4 + (end_quarter-start_quarter) + 1

df11['num_quarters'] = df11.apply(lambda row: quarter_difference(row["Earliest Quarter"],\
                                                                 row["Latest Quarter"]),axis=1)
df11.head(5)

,Bank,Earliest Quarter,Latest Quarter,Observation ID,Bonus,Deposit,Incentive,num_quarters
0,Ent Credit Union,2020Q1,2020Q1,3,300,100,1,1
1,Ent Credit Union,2020Q1,2020Q1,5,200,10000,1,1
2,Ent Credit Union,2020Q4,2021Q1,9,100,0,1,2
3,Ent Credit Union,2021Q4,2021Q4,21,200,0,1,1
4,Ent Credit Union,2022Q3,2022Q3,23,200,100,1,1


In [12]:
def next_quarter(date):
    year, quarter = date.split("Q")
    year, quarter = int(year), int(quarter)
    quarter += 1
    if quarter > 4:
        quarter = 1
        year += 1
    return f"{year}Q{quarter}"

In [13]:
# Create a list of quarter dates by looping over the start_quarter and num_quarters
quarter_dates = []
observation_id = []

for idx, row in df11.iterrows():
    start_quarter = row["Earliest Quarter"]
    num_quarters = row["num_quarters"]
    oid = row["Observation ID"]
    observation_id.append(oid)
    quarters = [start_quarter]
    for i in range(num_quarters - 1):
        start_quarter = next_quarter(start_quarter)
        quarters.append(start_quarter)
        observation_id.append(oid)
    quarter_dates.extend(quarters)

# Convert the list of quarter dates into a DataFrame
quarter_df = pd.DataFrame({"Observation ID":observation_id, "Quarter": quarter_dates})

In [14]:
# Merge with the original df1
final_df1 = quarter_df.merge(df11, on="Observation ID")
final_df1 = final_df1[["Bank","Quarter","Observation ID","Bonus","Deposit","Incentive"]]
final_df1

,Bank,Quarter,Observation ID,Bonus,Deposit,Incentive
0,Ent Credit Union,2020Q1,3,300,100,1
1,Ent Credit Union,2020Q1,5,200,10000,1
2,Ent Credit Union,2020Q4,9,100,0,1
3,Ent Credit Union,2021Q1,9,100,0,1
4,Ent Credit Union,2021Q4,21,200,0,1
...,...,...,...,...,...,...
15620,Bank Of Blue Valley,2019Q3,2286122,350,0,1
15621,Minnesota Bank & Trust,2019Q3,2286233,350,0,1
15622,Minnesota Bank & Trust,2019Q3,2286269,350,0,1
15623,Addition Financial,2019Q3,2286284,300,500,1


In [15]:
print("Range of quarters:", final_df1['Quarter'].min(), "to", final_df1['Quarter'].max())

Range of quarters: 2012Q3 to 2022Q4


### Data Cleaning Dataset B

In [16]:
df2 = pd.read_csv("DatasetB.csv")
df2.head(5)

,rssdid,chartertype,cert,bhcid,name,fullname,date,dateq,nbranch,hhi_depsumbr,...,timedep,timedepuninsured,unusedcomm,intexp,netinc,intincnet,nonintexp,nonintinc,operinc,domdepservicecharges
0,37,200,10057,0.0,BANK OF HANCOCK CTY,BANK OF HANCOCK COUNTY,20030331,2003q1,1.0,0.772909,...,38978.0,10630.0,1533.0,557.0,148.0,645.0,494.0,80.0,1282.0,64.0
1,37,200,10057,0.0,BANK OF HANCOCK CTY,BANK OF HANCOCK COUNTY,20030630,2003q2,1.0,0.772909,...,38869.0,10703.0,982.0,506.0,184.0,702.0,500.0,83.0,1291.0,72.0
2,37,200,10057,0.0,BANK OF HANCOCK CTY,BANK OF HANCOCK COUNTY,20030930,2003q3,1.0,0.772909,...,39268.0,10602.0,523.0,484.0,236.0,768.0,499.0,79.0,1331.0,67.0
3,37,200,10057,0.0,BANK OF HANCOCK CTY,BANK OF HANCOCK COUNTY,20031231,2003q4,1.0,0.772909,...,41391.0,12117.0,758.0,486.0,244.0,792.0,497.0,57.0,1335.0,80.0
4,37,200,10057,0.0,BANK OF HANCOCK CTY,BANK OF HANCOCK COUNTY,20040331,2004q1,1.0,0.772909,...,42420.0,12350.0,849.0,496.0,175.0,735.0,560.0,98.0,1329.0,69.0


In [17]:
df2["Bank"] = df2["fullname"].str.lower()
df2["Bank"] = df2["Bank"].str.title()
df2["Quarter"] = df2["dateq"].str.title()
df2 = df2[["Bank","Quarter","assets","equity","demanddep","deposits",\
           "intincnet","nonintinc","netinc"]].rename(columns={"intincnet":"int_income",
                                                            "nonintinc":"nonint_income",
                                                            "netinc":"net_income"})
df2

,Bank,Quarter,assets,equity,demanddep,deposits,int_income,nonint_income,net_income
0,Bank Of Hancock County,2003Q1,75143.0,12709.0,6219.0,62083.0,645.0,80.0,148.0
1,Bank Of Hancock County,2003Q2,74575.0,12796.0,5684.0,61391.0,702.0,83.0,184.0
2,Bank Of Hancock County,2003Q3,75996.0,13033.0,5935.0,62481.0,768.0,79.0,236.0
3,Bank Of Hancock County,2003Q4,78570.0,13180.0,5722.0,65146.0,792.0,57.0,244.0
4,Bank Of Hancock County,2004Q1,81219.0,13355.0,6622.0,67556.0,735.0,98.0,175.0
...,...,...,...,...,...,...,...,...,...
716728,Rockpoint Bank National Association,2021Q1,40468.0,30328.0,2936.0,9764.0,0.0,0.0,-826.0
716729,Rockpoint Bank National Association,2021Q2,71955.0,29243.0,5018.0,42458.0,65.0,2.0,-1128.0
716730,Square Fs,2021Q1,66225.0,55830.0,0.0,5011.0,5.0,1367.0,-175.0
716731,Square Fs,2021Q2,125430.0,61402.0,0.0,53057.0,2004.0,14161.0,4529.0


In [18]:
print("Range of quarters:", df2["Quarter"].min(), "to", df2["Quarter"].max())

Range of quarters: 1994Q1 to 2021Q2


In [19]:
# Subset df2 based on the quarter and drop duplicates
df2 = df2.loc[df2["Quarter"]>="2012Q3"]
df2 = df2.drop_duplicates(subset=["Bank","Quarter"], keep="first")

In [20]:
# compute various statistics and drop unused variables
df2["ROA"] = df2["net_income"]/df2["assets"]*100
df2["ROE"] = df2["net_income"]/df2["equity"]*100
df2["int_margin"] = df2["int_income"]/df2["deposits"]*100
df2["nonint_pct"] = df2["nonint_income"]/df2["net_income"]
df2 = df2.drop(columns=["assets","equity","net_income","int_income","nonint_income"])

In [21]:
# check if all banks deposits are above the requirement
df2["DepReq"] = np.where(df2["deposits"]>=df2["demanddep"], 1, 0)
df2[df2["DepReq"]==0]
df2 = df2.drop(columns=["demanddep","deposits","DepReq"])

In [22]:
final_df2 = df2.copy().dropna()
final_df2

,Bank,Quarter,ROA,ROE,int_margin,nonint_pct
38,Bank Of Hancock County,2012Q3,0.092391,0.411679,1.079120,0.921053
39,Bank Of Hancock County,2012Q4,0.211946,0.984955,1.054865,0.351648
40,Bank Of Hancock County,2013Q1,0.099704,0.471647,0.936291,0.735632
41,Bank Of Hancock County,2013Q2,0.264428,1.336150,0.926109,0.658120
42,Bank Of Hancock County,2013Q3,0.241248,1.214340,1.135490,0.373206
...,...,...,...,...,...,...
716726,Classic City Bank,2021Q2,-0.292596,-1.735771,0.677989,-0.021944
716728,Rockpoint Bank National Association,2021Q1,-2.041119,-2.723556,0.000000,-0.000000
716729,Rockpoint Bank National Association,2021Q2,-1.567646,-3.857333,0.153092,-0.001773
716730,Square Fs,2021Q1,-0.264251,-0.313452,0.099780,-7.811429


### Combine 2 Datasets

In [23]:
# Add Fed Fund Rate Dataset

# download the data from fed website
fed = pd.read_csv("FEDFUNDS.csv")

fed["Quarter"] = pd.to_datetime(fed["DATE"]).dt.to_period("Q")
fed.drop_duplicates(subset="Quarter", keep="last", inplace=True)
fed = fed.drop(["DATE"],axis=1).reset_index(drop=True).rename(columns={"FEDFUNDS":"Interest Rate"})
fed["Quarter"] = fed["Quarter"].astype(str)
fed

,Interest Rate,Quarter
0,1.07,1954Q3
1,1.28,1954Q4
2,1.35,1955Q1
3,1.64,1955Q2
4,2.18,1955Q3
...,...,...
270,0.20,2022Q1
271,1.21,2022Q2
272,2.56,2022Q3
273,4.10,2022Q4


In [24]:
df = final_df1.merge(final_df2, on=["Bank","Quarter"])
df = df.merge(fed, on=["Quarter"])
df

,Bank,Quarter,Observation ID,Bonus,Deposit,Incentive,ROA,ROE,int_margin,nonint_pct,Interest Rate
0,Bank Of America,2019Q4,94,500,10000,1,0.339561,2.965854,0.877300,0.611729,1.55
1,Bank Of America,2019Q4,118,300,4000,1,0.339561,2.965854,0.877300,0.611729,1.55
2,Bank Of America,2019Q4,121,200,20000,1,0.339561,2.965854,0.877300,0.611729,1.55
3,Bank Of America,2019Q4,622,350,0,1,0.339561,2.965854,0.877300,0.611729,1.55
4,Bank Of America,2019Q4,710,300,4000,1,0.339561,2.965854,0.877300,0.611729,1.55
...,...,...,...,...,...,...,...,...,...,...,...
6319,New York Commercial Bank,2016Q1,2216764,25,0,1,0.200386,1.264198,1.047814,0.391327,0.36
6320,New York Commercial Bank,2016Q1,2216765,25,0,1,0.200386,1.264198,1.047814,0.391327,0.36
6321,New York Commercial Bank,2016Q1,2216770,25,0,1,0.200386,1.264198,1.047814,0.391327,0.36
6322,New York Commercial Bank,2016Q1,2216818,150,0,1,0.200386,1.264198,1.047814,0.391327,0.36


In [25]:
df3 = df.drop_duplicates(subset=["Bank","Quarter"], keep="first").drop(columns=["Bonus","Deposit","Observation ID"])
df3

,Bank,Quarter,Incentive,ROA,ROE,int_margin,nonint_pct,Interest Rate
0,Bank Of America,2019Q4,1,0.339561,2.965854,0.877300,0.611729,1.55
11,Citibank,2019Q4,1,0.299932,2.904971,2.012814,0.815868,1.55
34,Pnc,2019Q4,1,0.258736,2.462611,0.886009,1.499893,1.55
42,Glacier Bank,2019Q4,1,0.448247,2.970853,1.293855,0.411852,1.55
43,Chase,2019Q4,1,1.019558,1.027956,76.200000,0.000000,1.55
...,...,...,...,...,...,...,...,...
6312,Farmington Bank,2016Q1,1,0.146016,1.848017,0.819307,0.735294,0.36
6313,Astoria Bank,2016Q1,1,0.144091,1.179735,0.934281,0.445753,0.36
6316,West Shore Bank,2016Q1,1,0.179900,1.972556,1.078097,1.040673,0.36
6319,New York Commercial Bank,2016Q1,1,0.200386,1.264198,1.047814,0.391327,0.36


## Model 1: Find factors that determine whether the bank offers incentives
### 1

In [26]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.discrete.discrete_model import Logit

x1 = df3[["ROA","ROE","int_margin","nonint_pct","Interest Rate"]]
y = df3["Incentive"]

# Run logistic regression
out = sm.Logit(y, x1).fit()
print(out.summary())

Optimization terminated successfully.
         Current function value: 0.545065
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:              Incentive   No. Observations:                 1684
Model:                          Logit   Df Residuals:                     1679
Method:                           MLE   Df Model:                            4
Date:                Fri, 03 Feb 2023   Pseudo R-squ.:                -0.02218
Time:                        21:05:11   Log-Likelihood:                -917.89
converged:                       True   LL-Null:                       -897.97
Covariance Type:            nonrobust   LLR p-value:                     1.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
ROA               2.8064      0.974      2.882      0.004       0.898       4.715
ROE              -0

From the output, only the coefficient for "Interest Margin" is significant.

In [27]:
def prob(beta, x):
    b0 = 0
    p = 1 - np.exp(b0-beta*x)/(1 + np.exp(b0-beta*x))
    return p

In [28]:
prob_incentive = prob(out.params.values, 1)
incentive = pd.DataFrame(prob_incentive*100)
incentive.rename(index={'1': 'incentive'})
print(incentive)

           0
0  94.302099
1  46.481826
2  64.906529
3  50.376531
4  49.506099


### 2

In [29]:
# create dummy variable for ROA and ROE
industry_avg = df3.groupby("Quarter")["ROA"].mean()
df3 = df3.merge(industry_avg, on="Quarter")
df3 = df3.rename(columns={"ROA_x":"ROA", "ROA_y":"ROA_avg"})
df3["ROA_d"] = np.where(df3["ROA"]>=df3["ROA_avg"], 1, 0)

industry_avg = df3.groupby("Quarter")["ROE"].mean()
df3 = df3.merge(industry_avg, on="Quarter")
df3 = df3.rename(columns={"ROE_x":"ROE", "ROE_y":"ROE_avg"})
df3["ROE_d"] = np.where(df3["ROE"]>=df3["ROE_avg"], 1, 0)

In [30]:
df3

,Bank,Quarter,Incentive,ROA,ROE,int_margin,nonint_pct,Interest Rate,ROA_avg,ROA_d,ROE_avg,ROE_d
0,Bank Of America,2019Q4,1,0.339561,2.965854,0.877300,0.611729,1.55,0.443601,0,2.421920,1
1,Citibank,2019Q4,1,0.299932,2.904971,2.012814,0.815868,1.55,0.443601,0,2.421920,1
2,Pnc,2019Q4,1,0.258736,2.462611,0.886009,1.499893,1.55,0.443601,0,2.421920,1
3,Glacier Bank,2019Q4,1,0.448247,2.970853,1.293855,0.411852,1.55,0.443601,1,2.421920,1
4,Chase,2019Q4,1,1.019558,1.027956,76.200000,0.000000,1.55,0.443601,1,2.421920,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1679,Farmington Bank,2016Q1,1,0.146016,1.848017,0.819307,0.735294,0.36,0.243155,0,2.191129,0
1680,Astoria Bank,2016Q1,1,0.144091,1.179735,0.934281,0.445753,0.36,0.243155,0,2.191129,0
1681,West Shore Bank,2016Q1,1,0.179900,1.972556,1.078097,1.040673,0.36,0.243155,0,2.191129,0
1682,New York Commercial Bank,2016Q1,1,0.200386,1.264198,1.047814,0.391327,0.36,0.243155,0,2.191129,0


In [31]:
x2 = df3[["ROA_d","ROE_d","int_margin","nonint_pct","Interest Rate"]]
y = df3["Incentive"]

# Run logistic regression
out = sm.Logit(y, x2).fit()
print(out.summary())

Optimization terminated successfully.
         Current function value: 0.544681
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:              Incentive   No. Observations:                 1684
Model:                          Logit   Df Residuals:                     1679
Method:                           MLE   Df Model:                            4
Date:                Fri, 03 Feb 2023   Pseudo R-squ.:                -0.02146
Time:                        21:05:13   Log-Likelihood:                -917.24
converged:                       True   LL-Null:                       -897.97
Covariance Type:            nonrobust   LLR p-value:                     1.000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
ROA_d            -0.0868      0.156     -0.556      0.579      -0.393       0.219
ROE_d             0

In [32]:
prob_incentive = prob(out.params.values, 1)
incentive = pd.DataFrame(prob_incentive*100)
incentive.rename(index={'1': 'incentive'})
print(incentive)

           0
0  47.832219
1  65.094210
2  67.702331
3  50.489151
4  50.245684


Interpretations:

1. According to the output, the small p value shows that the coefficients for "ROE_d" and "int_margin" are significant.

2. It can be concluded that Net Interest Margin and whether the ROE is higher than industry average affect the banks' decision whether to give out incentives.

3. If the bank's ROE is higher than the industry average, the probability of it offering incentives is as high as 65%.


## Model 2: Find factors that affect the intensity of incentives offered by banks

In [33]:
df4 = df.copy()
incentive_num = df4.groupby(["Bank","Quarter"])["Incentive"].sum()
df4 = df4.merge(incentive_num, on=["Bank","Quarter"])
df4 = df4.rename(columns={"Incentive_y":"Incentive_num"})
df4.drop_duplicates(subset=["Bank","Quarter"], keep="first", inplace=True)
df4 = df4.drop(columns=["Observation ID","Incentive_x"],axis=1)
df4

,Bank,Quarter,Bonus,Deposit,ROA,ROE,int_margin,nonint_pct,Interest Rate,Incentive_num
0,Bank Of America,2019Q4,500,10000,0.339561,2.965854,0.877300,0.611729,1.55,11
11,Citibank,2019Q4,400,15000,0.299932,2.904971,2.012814,0.815868,1.55,20
34,Pnc,2019Q4,500,5000,0.258736,2.462611,0.886009,1.499893,1.55,8
42,Glacier Bank,2019Q4,25,0,0.448247,2.970853,1.293855,0.411852,1.55,1
43,Chase,2019Q4,200,15000,1.019558,1.027956,76.200000,0.000000,1.55,61
...,...,...,...,...,...,...,...,...,...,...
6312,Farmington Bank,2016Q1,100,0,0.146016,1.848017,0.819307,0.735294,0.36,1
6313,Astoria Bank,2016Q1,250,0,0.144091,1.179735,0.934281,0.445753,0.36,3
6316,West Shore Bank,2016Q1,25,0,0.179900,1.972556,1.078097,1.040673,0.36,3
6319,New York Commercial Bank,2016Q1,25,0,0.200386,1.264198,1.047814,0.391327,0.36,4


In [34]:
y = df4["Incentive_num"]
x1 = df4[["ROA","ROE","int_margin","nonint_pct","Interest Rate"]]

model = sm.OLS(y, x1)
out = model.fit()
print(out.summary())

                                 OLS Regression Results                                
Dep. Variable:          Incentive_num   R-squared (uncentered):                   0.174
Model:                            OLS   Adj. R-squared (uncentered):              0.172
Method:                 Least Squares   F-statistic:                              70.77
Date:                Fri, 03 Feb 2023   Prob (F-statistic):                    2.55e-67
Time:                        21:05:14   Log-Likelihood:                         -5725.0
No. Observations:                1684   AIC:                                  1.146e+04
Df Residuals:                    1679   BIC:                                  1.149e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

Interpretation:

1. Observed from the p-value, coefficients for "ROA", "ROE", and "Interest Rate" are sigfinicant. It might be concluded that these variables are the important factors affecting the number of incentive campaigns.

2. The R-squared is around 17%. There is no problem of over-fitting.

In [35]:
industry_avg = df4.groupby("Quarter")["ROA"].mean()
df4 = df4.merge(industry_avg, on="Quarter")
df4 = df4.rename(columns={"ROA_x":"ROA", "ROA_y":"ROA_avg"})
df4["exROA"] = df4["ROA"]-df4["ROA_avg"]

industry_avg = df4.groupby("Quarter")["ROE"].mean()
df4 = df4.merge(industry_avg, on="Quarter")
df4 = df4.rename(columns={"ROE_x":"ROE", "ROE_y":"ROE_avg"})
df4["exROE"] = df4["ROE"]-df4["ROE_avg"]

In [36]:
y = df4["Incentive_num"]
x2 = df4[["exROA","exROE","int_margin","nonint_pct","Interest Rate"]]

model = sm.OLS(y, x2)
out = model.fit()
print(out.summary())

                                 OLS Regression Results                                
Dep. Variable:          Incentive_num   R-squared (uncentered):                   0.163
Model:                            OLS   Adj. R-squared (uncentered):              0.161
Method:                 Least Squares   F-statistic:                              65.41
Date:                Fri, 03 Feb 2023   Prob (F-statistic):                    1.58e-62
Time:                        21:05:16   Log-Likelihood:                         -5736.2
No. Observations:                1684   AIC:                                  1.148e+04
Df Residuals:                    1679   BIC:                                  1.151e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

Interpretation:

1. With reference from the p_value, coefficients for "exROA","exROE","Int_margin",and "Interest rate" are significant.
2. The r-squared is not very large, indicating there is no over-fitting problem.
3. The empirical results matches with my assumptions. How much the bank overperforms the industry, the ability of generating income from deposits, and the macro environment are the most important factors that determine the intensity of incentives.
